# Segundo modelo de regressão logística

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-muted")
%matplotlib inline

# Lendo os dados de treino e teste

---

In [2]:
treino = pd.read_csv("train.csv")
teste = pd.read_csv("test.csv")

# Explorando os dados 

---

In [3]:
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


---

## Dicionário das variáveis

Podemos observar que temos as seguinte estrutura de variáveis no banco:

In [4]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


---

O que significa cada uma, segundo o [Kaggle](https://www.kaggle.com/c/titanic/data) é:

1. **PassengerId**: Um ID para identificar cada passageiro;
2. **Survived**: Se ele sobreviveu (0 = Não, 1 = Sim) - Qualitativa;
3. **Pclass**: Ticket que indica qual classe o passageiro estava (1 = 1st, 2 = 2nd, 3 = 3rd) - Qualitativa;
4. **Name**: Nome - Qualitativa;
5. **Sex**: Sexo - Qualitativa;
6. **Age**: Idade - Quantitativa;
7. **SibSp**: Nº de irmãos / cônjuges a bordo do Titanic - Quantitativa;
8. **Parch**: Nº de pais / filhos a bordo do Titanic - Quantitativa;
9. **Ticket**: Nº do ticket - Qualitativa;
10. **Fare**: Tarifa do passageiro - Quantitativa;
11. **Cabin**: Número da cabine - Qualitativa;
12. **Embarked**: Porto em que o passageiro embarcou - Qualitativa.

# Pré-processamento dos dados

---



## Concatenar os dados de treino e teste

In [5]:
survived = treino["Survived"]

survived

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [6]:
treino.drop(["Survived"], axis = 1, inplace = True)

treino_index = treino.shape[0]
teste_index = teste.shape[0]

banco_geral = pd.concat(objs = [treino, teste], axis = 0).reset_index(drop = True)

In [7]:
banco_geral.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
banco_geral.shape

(1309, 11)

## Selecionando as variáveis

In [9]:
banco_geral = banco_geral.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)

In [10]:
banco_geral.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


## Transformação de variáveis

1. *Age*: A idade será substituída por uma variável chamada Faixa Etária e os valores nulos terão uma classe SI.
2. *Pclass*: Recodificar ela para variável qualitativa.
3. *Fare*: Devido a distribuição da variável Fare possuir outliers, iremos utilizar o log.
4. *Embarked*: Aqui podemos criar uma categoria a mais também dizendo que não sabemos de qual porto a pessoa embarcou.
5. *SibSp + Parch*: Criamos uma variável chamada "Qtd_familiares", que diz o total de parentes que cada passageiro.
6. *Onehotencode* nas variáveis qualitativas

### Age

A idade será substituída por uma variável chamada Faixa Etária e os valores nulos terão uma classe SI.

In [11]:
banco_geral["Age"] = pd.cut(banco_geral["Age"], bins = [0, 20, 40, 60, 80], labels = ["0-20", "21-40", "41-60", "61-80"], include_lowest = True)

banco_geral["Age"] = banco_geral["Age"].values.add_categories("SI").fillna("SI")

--- 

Ao final temos a seguinte contagem para cada categoria.

In [12]:
banco_geral["Age"].value_counts()

21-40    571
SI       263
0-20     248
41-60    194
61-80     33
Name: Age, dtype: int64

### Pclass

Recodificar ela para variável qualitativa, já que no modelo de regressão logística ela foi como variável quantitativa.

In [13]:
banco_geral["Pclass"] = banco_geral.replace({'Pclass': {1: "Classe 1", 
                                                        2: "Classe 2", 
                                                        3: "Classe 3"}})

In [14]:
banco_geral["Pclass"].value_counts()

Classe 3    709
Classe 1    323
Classe 2    277
Name: Pclass, dtype: int64

### Fare

1. **Valores nulos**: Devido aos mínimos outliers que vimos nos gráficos do notebook anterior, podemos escolher as seguintes opções:

    - Moda: O input pela moda (valor da tarifa mais frequente, isto é vamos considerar que os passageiros que não possuem essa informação terão o valor de tarifa mais frequente);
    - Mediana: Input pela mediana (valor da tarifa que separa em 50%).
    
Nesse caso, escolhi por inputar a **moda** nos valores faltantes.

2. **Variação**: Como a distribuição da Tarifa possui valores super dispersos, por exemplo (**ver o notebook anterior em https://github.com/barbosarafael/Projetos/blob/master/Titanic%20-%20Kaggle/notebook_titanic_kaggle.ipynb**), optei pela seguinte estratégia:

     - Log;
     - Normalização;
     - Padronização.
    
     
Nesse caso, escolhi o log. 

**Entretanto**, temos um problema pois o log de 0 não é definido (ver este [link](https://www.rapidtables.com/math/algebra/logarithm/Logarithm_of_0.html)). Mas, sabemos que o $ log(1) = 0 $, então a solução é substituir todos os valores de 0 para 1 e, após isso, utilizar o log.

#### Tirando a moda dos dados

In [15]:
moda_fare = banco_geral["Fare"].mode()[0]

print("A moda das Tarifas é", moda_fare)

A moda das Tarifas é 8.05


#### Substituindo os valores faltantes pela moda

In [16]:
banco_geral["Fare"] = banco_geral["Fare"].fillna(moda_fare)

banco_geral["Fare"]

0         7.2500
1        71.2833
2         7.9250
3        53.1000
4         8.0500
          ...   
1304      8.0500
1305    108.9000
1306      7.2500
1307      8.0500
1308     22.3583
Name: Fare, Length: 1309, dtype: float64

#### Substituindo os valores 0 por 1

In [17]:
banco_geral = banco_geral.replace({"Fare": {0 : 1}})

#### Aplicando o log na variável 

In [18]:
banco_geral["Fare"] = np.log(banco_geral["Fare"])

banco_geral.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,Classe 3,male,21-40,1,0,1.981001,S
1,Classe 1,female,21-40,1,0,4.266662,C
2,Classe 3,female,21-40,0,0,2.070022,S
3,Classe 1,female,21-40,1,0,3.972177,S
4,Classe 3,male,21-40,0,0,2.085672,S


### Embarked

In [19]:
banco_geral["Embarked"] = banco_geral["Embarked"].fillna("SI")

In [20]:
banco_geral["Embarked"].value_counts()

S     914
C     270
Q     123
SI      2
Name: Embarked, dtype: int64

## SibSp + Parch

Criar uma variável chamada "Qtd_familiares", que diz o total de parentes que cada passageiro.

In [21]:
banco_geral["Qtd_familiares"] = banco_geral["SibSp"] + banco_geral["Parch"]

banco_geral["Qtd_familiares"]

banco_geral = banco_geral.drop(["SibSp"], axis = 1)

## OneHotEncode nas variáveis qualitativas/categóricas

In [22]:
banco_geral = pd.get_dummies(banco_geral)

In [23]:
banco_geral.head()

,Parch,Fare,Qtd_familiares,Pclass_Classe 1,Pclass_Classe 2,Pclass_Classe 3,Sex_female,Sex_male,Age_0-20,Age_21-40,Age_41-60,Age_61-80,Age_SI,Embarked_C,Embarked_Q,Embarked_S,Embarked_SI
0,0,1.981001,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0
1,0,4.266662,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0
2,0,2.070022,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0
3,0,3.972177,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0
4,0,2.085672,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0


## Separando em treino e teste novamente

In [24]:
treino1 = banco_geral.iloc[:treino_index]

print("Dimensões do novos dados de treino")

treino1.shape

Dimensões do novos dados de treino


(891, 17)

In [25]:
teste1 = banco_geral.iloc[:teste_index]

print("Dimensões do novos dados de teste")

teste1.shape

Dimensões do novos dados de teste


(418, 17)

In [26]:
treino1 = treino1.assign(Survived = survived)

treino1.head()

,Parch,Fare,Qtd_familiares,Pclass_Classe 1,Pclass_Classe 2,Pclass_Classe 3,Sex_female,Sex_male,Age_0-20,Age_21-40,Age_41-60,Age_61-80,Age_SI,Embarked_C,Embarked_Q,Embarked_S,Embarked_SI,Survived
0,0,1.981001,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0
1,0,4.266662,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1
2,0,2.070022,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1
3,0,3.972177,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1
4,0,2.085672,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0


## Separando os dados de treino em validação também

In [27]:
from sklearn.model_selection import train_test_split

x_treino, x_valid, y_treino, y_valid = train_test_split(treino1.drop("Survived", axis = 1), treino1["Survived"], train_size = 0.5, random_state = 1234)

In [28]:
print("Os dados de treino possui dimensões:", treino1.shape)
print("---")
print("x_treino possui dimensões:", x_treino.shape)
print("---")
print("y_treino possui dimensões:", y_treino.shape)
print("---")
print("x_valid possui dimensões:", x_valid.shape)
print("---")
print("y_valid possui dimensões:", y_valid.shape)

Os dados de treino possui dimensões: (891, 18)
---
x_treino possui dimensões: (445, 17)
---
y_treino possui dimensões: (445,)
---
x_valid possui dimensões: (446, 17)
---
y_valid possui dimensões: (446,)


# Regressão logística

---

Modificações nos parâmetros

1. **random_state**: Semente para reprodutibilidade;
2. **class_weight**: O peso que estou dando para cada classe. Como vimos no notebook anterior, a proporção de Sobreviventes era de, aproximadamente, 0.6, consequentemente, a de não sobreviventes era de 0.4. Modificar esse parâmetro fez o modelo passar a baseline em 0.015 (aproximadamente).

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression(random_state = 1234, class_weight = {0 : 0.6, 1 : 0.4})
logreg.fit(x_treino, y_treino)

LogisticRegression(C=1.0, class_weight={0: 0.6, 1: 0.4}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1234, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
previsao = logreg.predict(x_valid)

previsao[:6]

array([1, 0, 0, 1, 0, 0], dtype=int64)

## Matriz de confusão

In [31]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_valid, previsao)

print(confusion_matrix)

[[260  22]
 [ 60 104]]


## Acurácia nos dados de validação

O baseline a ser batida é: 0.8071748878923767

In [32]:
melhora_piora_modelo = logreg.score(x_valid, y_valid) - 0.8071748878923767

if melhora_piora_modelo > 0:
    
    print("Temos um ganho de", melhora_piora_modelo, "na acurácia.")

elif melhora_piora_modelo < 0:
    
    print("Temos uma perda de", melhora_piora_modelo, "na acurácia.")
    
else:
    
    print("Não temos ganho")

Temos um ganho de 0.008968609865470878 na acurácia.


# Retreinando o modelo (para ver o resultado no Kaggle)

---

In [33]:
x_treino_retrain = treino1.drop("Survived", axis = 1)
y_treino_retrain = treino1["Survived"]

logreg_retrain = LogisticRegression(random_state = 1234)

logreg_retrain.fit(x_treino_retrain, y_treino_retrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1234, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Prevendo para os dados de teste

In [34]:
previsoes_subm = logreg_retrain.predict(teste1)

previsoes_subm.shape

(418,)

In [35]:
id_teste = teste["PassengerId"]

subm = pd.DataFrame()

subm["PassengerId"] = id_teste

subm["Survived"] = previsoes_subm

In [36]:
subm.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,1
3,895,1
4,896,0


In [37]:
# subm.to_csv("subm6.csv", index = False)

# Conclusões

1. O modelo sem retreino teve acurácia de 0.58851 no Kaggle;
2. O modelo com retreino teve acurácia de 0.57416 no Kaggle.